In [1]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F

import wandb
import math

from ipywidgets import IntProgress
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import f1_score

import sys

from glob import glob
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
from time import time

import torch
import torch.utils.data as data

import matplotlib.pyplot as plt
import seaborn as sns
from torchvision import transforms
from torchvision.transforms import Resize, ToTensor, Normalize
import albumentations as A
import cv2
import albumentations.pytorch

import matplotlib.pyplot as plt
import seaborn as sns
import random
import csv 
import timm
import sklearn.metrics

In [6]:
device = torch.device("cuda:0")
batch_size = 16
class_n = 2
learning_rate = 1e-5 # 1e-4
epochs = 10
save_path = 'models/'

In [7]:
os.environ['WANDB_NOTEBOOK_NAME'] = 'code.ipynb'
wandb.init(project='NPDH', name='NB1D_final_data',
    config={
    "batch size": batch_size,
    "epochs" : epochs,
    "learning rate": learning_rate,
    # "backborn" : 'se_resnext101_32x4d', 
    # "architecture" : 'Deeplabv3+', 
})

In [8]:
def evaluation(ground_truth, pred_output, labels = [1,0]):
    mtris = sklearn.metrics.confusion_matrix(ground_truth,pred_output, labels =[1,0])
    TP, FN = mtris[0]
    FP, TN = mtris[1]
    ttl = TP+FN+FP+TN
    accuracy = (TP+TN)/ttl
    specificity = TN/(TN+FP)
    sensitivity = TP/(TP+FN)
    precision = TP/(TP+FP)
    negative_predicable_value = TN/(TN+FN)
    F1score = 2*precision*sensitivity/(precision+sensitivity)

    return accuracy,specificity,sensitivity,precision,negative_predicable_value,F1score

In [9]:
# Set random seed
import random
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
os.environ["PYTHONHASHSEED"] = str(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)  # type: ignore
torch.backends.cudnn.deterministic = True  # type: ignore
torch.backends.cudnn.benchmark = True  # type: ignore

In [10]:
# base_path = os.path.join(os.getcwd(),'a-trac-colon')
# colon_negative = 'colon_negative'
# colon_negative_test = 'colon_negative_test'
# colon_positive = 'colon_positive'
# colon_positive_test = 'colon_positive_test'

In [11]:
# train_negative_list = os.listdir(os.path.join(base_path,colon_negative))
# test_negative_list = os.listdir(os.path.join(base_path,colon_negative_test))
# train_positive_list = os.listdir(os.path.join(base_path,colon_positive))
# test_positive_list = os.listdir(os.path.join(base_path,colon_positive_test))

In [12]:
# train_nagative = [(img_name, os.path.join(base_path,colon_negative,img_name),0) for img_name in train_negative_list]
# train_positive = [(img_name, os.path.join(base_path,colon_positive,img_name),1) for img_name in train_positive_list]
# train_pd = pd.DataFrame(train_nagative+train_positive, columns = ['image_id','image_path','labels'])
# train_pd.to_csv('train_pd.csv')
# train_pd = train_pd.sample(frac=1).reset_index(drop=True)

In [13]:
# test_nagative = [(img_name, os.path.join(base_path,colon_negative,img_name),0) for img_name in test_negative_list]
# test_positive = [(img_name, os.path.join(base_path,colon_positive,img_name),1) for img_name in test_positive_list]
# test_pd = pd.DataFrame(test_nagative+test_positive, columns = ['image_id','image_path','labels'])
# test_pd.to_csv('test_pd.csv')
# test_pd = test_pd.sample(frac=1).reset_index(drop=True)

In [14]:
train_total = pd.read_csv("train_1_5.csv")
test = pd.read_csv("test_1_5.csv")

In [15]:
train_total

,Unnamed: 0,image_id,image_path,labels
0,0,Image_1.jpg,/opt/ml/nunbody/NHDP/a-trac-gi/final/gi_negati...,0
1,1,Image_100.jpg,/opt/ml/nunbody/NHDP/a-trac-gi/final/gi_negati...,0
2,2,Image_1000.jpg,/opt/ml/nunbody/NHDP/a-trac-gi/final/gi_negati...,0
3,3,Image_1001.jpg,/opt/ml/nunbody/NHDP/a-trac-gi/final/gi_negati...,0
4,4,Image_1002.jpg,/opt/ml/nunbody/NHDP/a-trac-gi/final/gi_negati...,0
...,...,...,...,...
6395,6395,Image_992.jpg,/opt/ml/nunbody/NHDP/a-trac-gi/final/gi_positi...,1
6396,6396,Image_994.jpg,/opt/ml/nunbody/NHDP/a-trac-gi/final/gi_positi...,1
6397,6397,Image_995.jpg,/opt/ml/nunbody/NHDP/a-trac-gi/final/gi_positi...,1
6398,6398,Image_997.jpg,/opt/ml/nunbody/NHDP/a-trac-gi/final/gi_positi...,1


## 데이터셋 구성

In [16]:
cv2.imread(train_total['image_path'][0]).shape

(512, 512, 3)

In [17]:
img = cv2.imread(train_total['image_path'][0])
img = np.transpose(img, (2,0,1))
img.shape

(3, 512, 512)

In [18]:
class CustomDataset(Dataset):
    def __init__(self, files, labels=None, mode='train'):
        self.mode = mode
        self.files = files
        if mode == 'train':
            self.labels = labels
            
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        if self.mode == 'train':
            img = cv2.imread(self.files[i])
            # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
            img = img.astype(np.float32)/255
            img = np.transpose(img, (2,0,1))
            return {
                'img' : torch.tensor(img, dtype=torch.float32),
                'label' : torch.tensor(self.labels[i], dtype=torch.long),
                'img_id' : self.files[i].split('/')[-1]
            }
        else:
            img = cv2.imread(self.files[i])
            # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.resize(img, dsize=(224, 224), interpolation=cv2.INTER_AREA)
            img = img.astype(np.float32)/255
            img = np.transpose(img, (2,0,1))
            return {
                'img' : torch.tensor(img, dtype=torch.float32),
                'img_id' : self.files[i].split('/')[-1]
            }

In [19]:
train = train_total
val = test

In [20]:
train_dataset = CustomDataset(train['image_path'], train['labels'].values)
val_dataset = CustomDataset(val['image_path'], val['labels'].values)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, num_workers=0, shuffle=True)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, num_workers=0, shuffle=False)

test_dataset = CustomDataset(test['image_path'], labels=None, mode='test')
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, num_workers=0, shuffle=False)

In [21]:
train_dataset[0]

{'img': tensor([[[0.9922, 0.9922, 0.9922,  ..., 0.8745, 0.7961, 0.7529],
          [0.9922, 0.9882, 0.9882,  ..., 0.8667, 0.8314, 0.7059],
          [0.9922, 0.9961, 0.9922,  ..., 0.9333, 0.8588, 0.6667],
          ...,
          [0.8235, 0.7843, 0.6863,  ..., 0.9725, 0.9490, 0.9373],
          [0.6706, 0.7490, 0.8275,  ..., 0.9569, 0.9490, 0.9373],
          [0.5765, 0.5922, 0.6667,  ..., 0.9412, 0.9255, 0.9059]],
 
         [[0.9922, 0.9922, 0.9922,  ..., 0.7294, 0.6235, 0.4667],
          [0.9922, 0.9882, 0.9882,  ..., 0.7529, 0.6941, 0.4549],
          [0.9922, 0.9961, 0.9922,  ..., 0.8392, 0.7098, 0.4039],
          ...,
          [0.6353, 0.6039, 0.4824,  ..., 0.9137, 0.8745, 0.8549],
          [0.4275, 0.5373, 0.6588,  ..., 0.9059, 0.8784, 0.8549],
          [0.1804, 0.2353, 0.4157,  ..., 0.8902, 0.8667, 0.8353]],
 
         [[0.9922, 0.9922, 0.9922,  ..., 0.8510, 0.7451, 0.5882],
          [0.9922, 0.9882, 0.9922,  ..., 0.8627, 0.8118, 0.5765],
          [0.9922, 0.9961, 0.9961

In [23]:
class DownsamplerBlock(nn.Module):
    def __init__(self, in_channels=32, out_channels=32):
        super(DownsamplerBlock, self).__init__()
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels-in_channels, kernel_size=3, stride=2, padding=1)
        self.maxpool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.bn = nn.BatchNorm2d(out_channels)

    def forward(self, input_tensor):
        x = torch.cat([self.conv(input_tensor), self.maxpool(input_tensor)], dim=1)
        x = self.bn(x)
        return F.relu(x)

class NB1D(nn.Module):
    def __init__(self, channels=32, dilation=1):
        super(NB1D, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=channels, out_channels=channels, kernel_size=(3, 1), stride=1,
                               padding=(1, 0))
        self.conv2 = nn.Conv2d(in_channels=channels, out_channels=channels, kernel_size=(1, 3), stride=1,
                               padding=(0, 1))

        self.bn1 = nn.BatchNorm2d(channels)
        self.conv3 = nn.Conv2d(in_channels=channels, out_channels=channels, kernel_size=(3, 1), stride=1,
                               padding=(1*dilation, 0), dilation=dilation)
        self.conv4 = nn.Conv2d(in_channels=channels, out_channels=channels, kernel_size=(1, 3), stride=1,
                               padding=(0, 1*dilation), dilation=dilation)
        self.bn2 = nn.BatchNorm2d(channels)

    def forward(self, input_tensor):
        x = self.conv1(input_tensor)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.bn1(x)
        x = F.relu(x)

        x = self.conv3(x)
        x = F.relu(x)
        x = self.conv4(x)
        x = self.bn2(x)
        return F.relu(x+input_tensor)

class ResNet50NB1D(nn.Module):
    def __init__(self, color_channel=3, num_classes=2):
        super(ResNet50NB1D, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=color_channel, out_channels=64, kernel_size=7, stride=2, padding=3)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.conv2_x = nn.Sequential(
            NB1D(channels=64),
            NB1D(channels=64),
            # NB1D(channels=64)
        )
        self.conv3_x = nn.Sequential(
            DownsamplerBlock(in_channels=64, out_channels=128),
            NB1D(channels=128),
            NB1D(channels=128),
            # NB1D(channels=128),
            # NB1D(channels=128)
        )
        self.conv4_x = nn.Sequential(
            DownsamplerBlock(in_channels=128, out_channels=256),
            NB1D(channels=256),
            NB1D(channels=256),
            # NB1D(channels=256),
            # NB1D(channels=256),
            # NB1D(channels=256),
            # NB1D(channels=256)
        )
        self.conv5_x = nn.Sequential(
            DownsamplerBlock(in_channels=256, out_channels=512),
            NB1D(channels=512),
            NB1D(channels=512),
            # NB1D(channels=512),
        )
        
        self.avg = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(256, num_classes)

    def forward(self, input_tensor):
        x = self.conv1(input_tensor)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.maxpool1(x)

        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.conv4_x(x)
        # x = self.conv5_x(x)
        x = self.avg(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x
model = ResNet50NB1D().to(device)

In [19]:

from efficientnet_pytorch import EfficientNet
model = EfficientNet.from_pretrained('efficientnet-b7',num_classes=2)
print("네트워크 필요 입력 채널 개수", model._conv_stem.weight.shape[1])
print("네트워크 출력 채널 개수 (예측 class type 개수)", model._fc.weight.shape[0])

model = model.to(device)

Downloading: "https://github.com/lukemelas/EfficientNet-PyTorch/releases/download/1.0/efficientnet-b7-dcc49843.pth" to /opt/ml/.cache/torch/hub/checkpoints/efficientnet-b7-dcc49843.pth


  0%|          | 0.00/254M [00:00<?, ?B/s]

Loaded pretrained weights for efficientnet-b7
네트워크 필요 입력 채널 개수 3
네트워크 출력 채널 개수 (예측 class type 개수) 2


In [20]:
# list(model.modules())

In [24]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min')

In [25]:
def evaluation(ground_truth, pred_output, labels = [1,0]):
    mtris = sklearn.metrics.confusion_matrix(ground_truth,pred_output, labels=labels)
    TP, FN = mtris[0]
    FP, TN = mtris[1]
    ttl = TP+FN+FP+TN
    accuracy = (TP+TN)/ttl
    specificity = TN/(TN+FP)
    sensitivity = TP/(TP+FN)
    precision = TP/(TP+FP)
    negative_predicable_value = TN/(TN+FN)
    F1score = 2*precision*sensitivity/(precision+sensitivity)

    return accuracy,specificity,sensitivity,precision,negative_predicable_value,F1score

def train_step(batch_item, epoch, batch, training):
    img = batch_item['img'].to(device)
    label = batch_item['label'].to(device)
    img_id = batch_item['img_id']
    ground_truth = []
    pred_output = []
    if training is True:
        model.train()
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            output = model(img)
            loss = criterion(output, label)
            pred_output+=output.argmax(axis =1).cpu().tolist()
            ground_truth+=label.cpu().tolist()
        loss.backward()
        optimizer.step()
        return loss,ground_truth,pred_output, img_id
    else:
        model.eval()
        with torch.no_grad():
            output = model(img)
            loss = criterion(output, label)
            pred_output+=output.argmax(axis =1).cpu().tolist()
            ground_truth+=label.cpu().tolist()
        return loss,ground_truth,pred_output, img_id



In [26]:
save_path = 'save_model/NB1D_final_data.pth'

In [28]:
loss_plot, val_loss_plot = [], []

for epoch in range(epochs):
    total_loss, total_val_loss = 0, 0
    
    tqdm_dataset = tqdm(enumerate(train_dataloader))
    training = True
    ground_truth = []
    pred_output = []
    
    for batch, batch_item in tqdm_dataset:
        batch_loss,gt,po,_ = train_step(batch_item, epoch, batch, training)
        total_loss += batch_loss
        ground_truth += gt
        pred_output += po
        tqdm_dataset.set_postfix({
            'Epoch': epoch + 1,
            'Loss': '{:06f}'.format(batch_loss.item()),
            'Total Loss' : '{:06f}'.format(total_loss/(batch+1))
        })
    loss_plot.append(total_loss/(batch+1))
    acc,spec,sens,prec,npv,f1 = evaluation(ground_truth,pred_output)

    wandb.log({
            'train Total Loss' : '{:06f}'.format(total_loss/(batch+1)),
            "train Accuracy" : acc, 
            "train Specificity" : spec,
            "train Sensitivity" : sens,
            "train Precision" : prec,
            "train Negative_Predicable_Value" : npv,
            "train F1score" : f1,
            "train total_mean" : (acc+spec+sens+prec+npv+f1)/6
        })
    
    tqdm_dataset = tqdm(enumerate(val_dataloader))
    training = False
    ground_truth = []
    pred_output = []   
    image_ids = []
    
    for batch, batch_item in tqdm_dataset:
        batch_loss,gt,po,ids = train_step(batch_item, epoch, batch, training)
        total_val_loss += batch_loss
        ground_truth+=gt
        pred_output+=po    
        image_ids+=ids
        tqdm_dataset.set_postfix({
            'Epoch': epoch + 1,
            'Val Loss': '{:06f}'.format(batch_loss.item()),
            'Total Val Loss' : '{:06f}'.format(total_val_loss/(batch+1))
        })
        
    acc,spec,sens,prec,npv,f1 = evaluation(ground_truth,pred_output)
    wandb.log({
            "val Total Loss": '{:06f}'.format(total_val_loss/(batch+1)),
            "val Accuracy" : acc, 
            "val Specificity" : spec,
            "val Sensitivity" : sens,
            "val Precision" : prec,
            "val Negative_Predicable_Value" : npv,
            "val F1score" : f1,
            "val total_mean" : (acc+spec+sens+prec+npv+f1)/6
    })
    val_loss_plot.append(total_val_loss/(batch+1))
    
    if min(val_loss_plot) == val_loss_plot[-1]:
        torch.save(model, save_path)

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

...

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

0it [00:00, ?it/s]

In [ ]:
with open("submission.csv", 'w') as file: 
    writer = csv.writer(file) 
    writer.writerow(pred_output)

In [ ]:
len(pred_output),len(ground_truth)

In [ ]:
evaluation(ground_truth,pred_output)

In [ ]:
test_dataset = CustomDataset(test['image_path'], labels=None, mode='test')
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=16, num_workers=0, shuffle=False)

# test_inference
tqdm_test_dataset = tqdm(enumerate(test_dataloader))
training = True
pred_output = []
ground_truth = []
for batch, batch_item in tqdm_test_dataset:
    model.eval()
    img = batch_item['img'].to(device)
    with torch.no_grad():
        output = model(img)
        pred_output.append(int(output.argmax().cpu()))
        ground_truth.append(val.labels.iloc[batch])

In [ ]:
output.argmax(axis =1).cpu().tolist()

In [ ]:
labels

In [ ]:
pred_output = [int(i.cpu())  for i in pred_output]

In [ ]:
TP, FN = mtris[0] 
FP, TN = mtris[1]

NameError: name 'mtris' is not defined

In [ ]:
FP, TN

(0, 1000)

In [ ]:
t = [1,0,1,0,0,0]
p = [1,1,0,0,0,0]
sklearn.metrics.confusion_matrix(t,p, labels =[1,0])

array([[1, 1],
       [1, 3]])